In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [6]:
df = pd.read_csv('../output/risultati_er.csv')

In [7]:
def geocode_comuni(df):

    # Create a copy to avoid modifying the original DataFrame
    df_geo = df.copy()
    
    # Initialize the geocoder
    geolocator = Nominatim(user_agent="my_geocoder")
    # Create a rate-limited version to avoid overwhelming the service
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    # Create empty columns for coordinates
    df_geo['latitude'] = None
    df_geo['longitude'] = None
    
    def get_coordinates(row):
        try:
            # Construct search query with comune and region for better accuracy
            search_query = f"{row['comune']}, {row['provincia']}, {row['regione']}, Italy"
            location = geocode(search_query)
            
            if location:
                return pd.Series([location.latitude, location.longitude])
            else:
                return pd.Series([None, None])
                
        except Exception as e:
            print(f"Error geocoding {row['comune']}: {str(e)}")
            return pd.Series([None, None])
    
    # Apply geocoding to each row
    print("Starting geocoding process...")
    coordinates = df_geo.apply(get_coordinates, axis=1)
    df_geo[['latitude', 'longitude']] = coordinates
    
    return df_geo


df_with_coordinates = geocode_comuni(df)

# Save results to avoid geocoding again
df_with_coordinates.to_csv('../output/viz/risultati_ER_w_coordinates.csv', index=False)

Starting geocoding process...
